# Install Libraries
Install information can be found [here](https://satijalab.org/seurat/articles/install.html)\
Vignette [here](https://satijalab.org/seurat/articles/pbmc3k_tutorial.html)

In [ ]:
#Seurat parameters
#rna_matrix = "path-to-matrix.h5" #h5 count matrix
rna_matrix = "~/Downloads/out.gene.bc.matrices.h5"
genome = "genome-name"#either hg38 or mm10

min_features = 200 #Seurat QC for number of min features
percent_MT = 5 #Seurat QC for max % of mt 
min_cells = 3 #Seurat QC for min number of cells

normalization_method = "LogNormalize"
normalization_scale_factor = 10000

variable_features_method = "vst"
variable_features_num = 2000

dim_loadings_dim = 2 #Number of dimensions to display

jackstraw_replicates = 100 #Number of replicate samplings to perform
jackstraw_score_dim = 20 #Which dimensions to examine, in the default case, 1:jackstraw_score_dim = 1:20
jackstraw_plot_dim = 15 #Dims to plot, in the default case, 1:jackstraw_plot_dim = 1:15

heatmap_dim = 1 #Dimensions to plot
heatmap_cells = 500 #A list of cells to plot. If numeric, just plots the top cells.
heatmap_balanced = TRUE #Plot an equal number of genes with both + and - scores.

umap_dim = 10 #Dimensions (number of PCs) used to create umap, in the default case, 1:umap_dim = 1:10
umap_resolution = 0.5 #Value of the resolution parameter, use a value below 1.0 if you want to obtain a smaller number of communities.

prefix = "prefix" #project name

#Terra specific parameters
table_name = "demux_BH3KTLDMXY"
experiment_name = "gm12878_fresh_RNA"

#Papermill specific parameters
papermill = TRUE

#jupyter notebook plot sizes
options(repr.plot.width=20, repr.plot.height=10)

In [ ]:
papermill <- as.logical(papermill)
heatmap_balanced <- as.logical(heatmap_balanced)

In [ ]:
if (!requireNamespace("Seurat", quietly = TRUE))
    install.packages("Seurat")
if (!requireNamespace("hdf5r", quietly = TRUE))
    install.packages("hdf5r")
if (!requireNamespace("future", quietly = TRUE))
    install.packages("future")
if (!requireNamespace("logr", quietly = TRUE))
    install.packages("logr")
if (!requireNamespace("grid", quietly = TRUE))
    install.packages("grid")
if (!requireNamespace("gridExtra", quietly = TRUE))
    install.packages("gridExtra")
if (!requireNamespace("ggplot2", quietly = TRUE))
    install.packages("ggplot2")

suppressMessages(library(hdf5r))
suppressMessages(library(Seurat))
suppressMessages(library(future))
suppressMessages(library(logr))
suppressMessages(library(grid))
suppressMessages(library(gridExtra))
suppressMessages(library(ggplot2))
future.seed=TRUE
plan("multisession")
options("logr.notes" = FALSE)
set.seed(1234)

In [ ]:
#Function to save plots
dir.create("plots", showWarnings=F)
printPNG <- function(name, plotObject, papermill, wf=10, hf=10){
    filename = paste0(prefix,".rna.seurat.",name,".",genome)
    if(papermill){
        ggsave(plot = plotObject,filename = sprintf("plots/%s.png", filename), width = wf, height = hf)
    }
}

#Function to create plots
create_plot = function(plot_list, title, subtitle, heights=unit(c(1,1,8), rep("in",3)), width = 7){
    g = c(list(title),list(subtitle),plot_list)
    N = length(plot_list)
    laym = rbind(rep(1,N),rep(2,N),(3:(N+2)))
    widths = unit(rep(width, length(plot_list)), rep("in",length(plot_list)))
    obj = arrangeGrob(grobs=g,layout_matrix=laym, heights=heights, widths = widths)
    return(obj)
}

#Create log file
logfile <- file.path(paste0(prefix,".rna.seurat.logfile.",genome,".txt"))
lf <- log_open(logfile)


In [ ]:
get_file <- function(path){
    dest <- getwd()
    gsutil_cp(path, dest)
    name <- basename(path)
    return(name)
}

if (!papermill){
    table <- avtable(table_name)
    rna_matrix <- get_file(table$h5_matrix[table[, sprintf('%s_id', table_name)] == experiment_name])
}

In [ ]:
#Read h5 matrix

data = tryCatch({
        log_print("# Reading h5 matrix")
    
        #Code start to read rna_matrix
        data = Read10X_h5(rna_matrix)
    
        #Code end to read rna_matrix
    
        log_print("SUCCESSFUL: Reading h5 matrix")
        return(data)
    },
    error = function(cond) {
        log_print("ERROR: Reading h5 matrix")
        log_print(cond)
    },
    warning = function(cond) {
        log_print("WARNING: Reading h5 matrix")
        log_print(cond)
        return(Read10X_h5(rna_matrix))
    }
)


In [ ]:
#Create Seurat Object

rna = tryCatch({
        log_print("# Create Seurat Object")
    
        #Code start to create seurat object
    
        rna <- CreateSeuratObject(counts = data, project = prefix, min.cells = 0, min.features = 0)
    
        #Code end to create seurat object
    
        log_print("SUCCESSFUL: Create Seurat Object")
        return(rna)
    },
    error = function(cond) {
        log_print("ERROR: Create Seurat Object")
        log_print(cond)
    }
)

In [ ]:
#Pre-filtered violin QC plots 

tryCatch({
        log_print("# Pre-filtered Violin Plot")
    
        #Code start to create pre-filtered rna QC violin plot
    
        rna[["percent.mt"]] <- PercentageFeatureSet(rna, pattern = "^MT-")
    
        obj <- VlnPlot(rna, features = c("nFeature_RNA", "nCount_RNA", "percent.mt"), ncol = 3)
        tg <- textGrob('Pre-filtered RNA QC plots', gp = gpar(fontsize = 25, fontface = 'bold', col = 'red'))
        sg <- textGrob(paste0("Library: ", prefix, "\n", 
                        nrow(x = rna), " genes across ", ncol(x = rna), 
                        " cells \n------------------------------------ \n"), 
                        gp = gpar(fontsize = 18, fontface = 'bold'))

        plot_list = list(obj[[1]], obj[[2]], obj[[3]])
        
        obj = create_plot(plot_list, tg, sg)
        grid.draw(obj)
        printPNG(name = "prefiltered_violin", plotObject = obj, papermill = papermill)
    
        #Code end to create violin plot
    
        log_print("SUCCESSFUL: Pre-filtered Violin Plot")
    },
    error = function(cond) {
        log_print("ERROR: Pre-filtered Violin Plot")
        log_print(cond)
    }
)

In [ ]:
# Create feature plots

tryCatch({
        log_print("# Pre-filtered QC Scatterplots")
    
        #Code start to create feature plots
    
        plot1 <- FeatureScatter(rna, feature1 = "nCount_RNA", feature2 = "percent.mt")
        plot2 <- FeatureScatter(rna, feature1 = "nCount_RNA", feature2 = "nFeature_RNA")
        
        tg <- textGrob('Pre-filtered QC scatterplots', gp = gpar(fontsize = 25, fontface = 'bold', col = 'red'))
        sg <- textGrob(paste0("Library: ", prefix, "\n", 
                        nrow(x = rna), " genes across ", ncol(x = rna), 
                        " cells \n------------------------------------ \n"), 
                        gp = gpar(fontsize = 18, fontface = 'bold'))

        plot_list = list(plot1,plot2)
        obj = create_plot(plot_list, tg, sg ,heights = unit(c(1,1,7), rep("in",3)), width = 5)
        grid.draw(obj)
        printPNG(name = "prefiltered_qc_scatterplots", plotObject = obj, papermill = papermill)
    
        #Code end to create feature plots
    
        log_print("SUCCESSFUL: Pre-filtered QC Scatterplots")
    },
    error = function(cond) {
        log_print("ERROR: Pre-filtered QC Scatterplots")
        log_print(cond)
    }
)

In [ ]:
#Subset Seurat Object by QC filters

rna = tryCatch({
        log_print("# Subset Seurat Object")
    
        #Code start to create seurat object
    
        rna <- CreateSeuratObject(counts = data, project = prefix, min.cells = min_cells, min.features = min_features)
        rna[["percent.mt"]] <- PercentageFeatureSet(rna, pattern = "^MT-")
        rna <- subset(rna, subset = percent.mt < percent_MT)
    
        #Code end to create seurat object
    
        log_print("SUCCESSFUL: Subset Seurat Object")
        return(rna)
    },
    error = function(cond) {
        log_print("ERROR: Subset Seurat Object")
        log_print(cond)
    }
)

In [ ]:
#Post-filtered violin QC plots 

tryCatch({
        log_print("# Post-filtered Violin Plot")
    
        #Code start to create pre-filtered rna QC violin plot
        
        obj <- VlnPlot(rna, features = c("nFeature_RNA", "nCount_RNA", "percent.mt"), ncol = 3)
        tg <- textGrob('Post-filtered RNA QC plots', gp = gpar(fontsize = 25, fontface = 'bold', col = 'red'))
        sg <- textGrob(paste0("Library: ", prefix, "\n", 
                        nrow(x = rna), " genes across ", ncol(x = rna), 
                        " cells \n------------------------------------ \n"), 
                        gp = gpar(fontsize = 18, fontface = 'bold'))

        plot_list = list(obj[[1]], obj[[2]], obj[[3]])
        obj = create_plot(plot_list, tg, sg)
        grid.draw(obj)
        printPNG(name = "postfiltered_violin", plotObject = obj, papermill = papermill)
        
        #Code end to create violin plot
    
        log_print("SUCCESSFUL: Post-filtered Violin Plot")
    },
    error = function(cond) {
        log_print("ERROR: Post-filtered Violin Plot")
        log_print(cond)
    }
)

In [ ]:
# Create feature plots

tryCatch({
        log_print("# Post-filtered QC Scatterplots")
    
        #Code start to create feature plots
    
        plot1 <- FeatureScatter(rna, feature1 = "nCount_RNA", feature2 = "percent.mt")
        plot2 <- FeatureScatter(rna, feature1 = "nCount_RNA", feature2 = "nFeature_RNA")
        
        tg <- textGrob('Post-filtered QC scatterplots', gp = gpar(fontsize = 25, fontface = 'bold', col = 'red'))
        sg <- textGrob(paste0("Library: ", prefix, "\n", 
                        nrow(x = rna), " genes across ", ncol(x = rna), 
                        " cells \n------------------------------------ \n"), 
                        gp = gpar(fontsize = 18, fontface = 'bold'))

        plot_list = list(plot1,plot2)
        obj = create_plot(plot_list, tg, sg, heights = unit(c(1,1,7), rep("in",3)))
        grid.draw(obj)
        printPNG(name = "postfiltered_qc_scatterplots", plotObject = obj, papermill = papermill)
        
        #Code end to create feature plots
    
        log_print("SUCCESSFUL: Post-filtered QC Scatterplots")
    },
    error = function(cond) {
        log_print("ERROR: Post-filtered QC Scatterplots")
        log_print(cond)
    }
)

In [ ]:
# Normalization

tryCatch({
        log_print("Normalization")
    
        #Code start to normalize
    
        rna <- NormalizeData(rna, normalization.method = normalization_method, scale.factor = normalization_scale_factor)
    
        #Code end to normalize
    
        log_print("SUCCESSFUL: Normalization")
    },
    error = function(cond) {
        log_print("ERROR: Normalization")
        log_print(cond)
    }
)


In [ ]:
# Find Variable Features

tryCatch({
        log_print("Finding Variable Features")
        
        #Code start to find variable features
    
        rna <- FindVariableFeatures(rna, selection.method = variable_features_method, nfeatures = variable_features_num)
        
        # Identify the 10 most highly variable genes
        top10 <- head(VariableFeatures(rna), 10)

        # plot variable features with and without labels
        #plot1 <- VariableFeaturePlot(rna)
        plot2 <- LabelPoints(plot = plot1, points = top10, repel = TRUE)
    
        tg <- textGrob('Variable Genes: Mean Expression-by-Variance Scatterplot', gp = gpar(fontsize = 25, fontface = 'bold', col = 'red'))
        sg <- textGrob(paste0("Library: ", prefix, "\n", 
                        nrow(x = rna), " genes across ", ncol(x = rna), 
                        " cells \n------------------------------------ \n"), 
                        gp = gpar(fontsize = 18, fontface = 'bold'))

        plot_list = list(plot2)
        obj = create_plot(plot_list, tg, sg, width = 8)
        grid.draw(obj)
        printPNG(name = "variable_genes", plotObject = obj, papermill = papermill)
        
        log_print("SUCCESSFUL: Finding Variable Features")
    },
    error = function(cond) {
        log_print("ERROR: Finding Variable Features")
        log_print(cond)
    }
)

In [ ]:
# Scaling

tryCatch({
        log_print("Scaling")
    
        #Code start to scale
    
        all.genes <- rownames(rna)
        rna <- ScaleData(rna, features = all.genes)
    
        #Code end to scale
    
        log_print("SUCCESSFUL: Scaling")
    },
    error = function(cond) {
        log_print("ERROR: Scaling")
        log_print(cond)
    }
)


In [ ]:
#Principal Component Analysis

tryCatch({
        log_print("Principal Component Analysis")

        # Code start to run PCA    
    
        rna <- RunPCA(rna, features = VariableFeatures(object = rna))
        obj <- VizDimLoadings(rna, dims = 1:dim_loadings_dim, reduction = "pca")
        
        tg <- textGrob('PCA: Dimension Loadings', gp = gpar(fontsize = 25, fontface = 'bold', col = 'red'))
        sg <- textGrob(paste0("Library: ", prefix, "\n", 
                        nrow(x = rna), " genes across ", ncol(x = rna), 
                        " cells \n------------------------------------ \n"), 
                        gp = gpar(fontsize = 18, fontface = 'bold'))
        plot_list = list(obj[[1]], obj[[2]])
        obj = create_plot(plot_list, tg, sg, width = 5)
        grid.draw(obj)
        printPNG(name = "pca_dimLoadings", plotObject = obj, papermill = papermill)
            
        # Code end to run PCA  

        log_print("SUCCESSFUL: Principal Component Analysis")
    },
    error = function(cond) {
        log_print("ERROR: Principal Component Analysis")
        log_print(cond)
    }
)

In [ ]:
# PCA plot

tryCatch({
        log_print("PCA plot")

        # Code start to create PCA plot 
    
        obj <- DimPlot(rna, reduction = "pca")
    
        tg <- textGrob('PCA Plot', gp = gpar(fontsize = 25, fontface = 'bold', col = 'red'))
        sg <- textGrob(paste0("Library: ", prefix, "\n", 
                        nrow(x = rna), " genes across ", ncol(x = rna), 
                        " cells \n------------------------------------ \n"), 
                        gp = gpar(fontsize = 18, fontface = 'bold'))
        plot_list = list(obj[[1]])
        obj = create_plot(plot_list, tg, sg, width = 8)
        grid.draw(obj)
        printPNG(name = "pca", plotObject = obj, papermill = papermill)
        
        # Code end to create PCA plot  

        log_print("SUCCESSFUL: PCA plot")
    },
    error = function(cond) {
        log_print("ERROR: PCA plot")
        log_print(cond)
    }
)

In [ ]:
# Heatmap

tryCatch({
        log_print("Heatmap")

        # Code start to create heatmap 
    
        obj <- DimHeatmap(rna, dims = heatmap_dim, cells = heatmap_cells, fast = FALSE, balanced = heatmap_balanced)
        tg <- textGrob('Heatmap', gp = gpar(fontsize = 25, fontface = 'bold', col = 'red'))
        sg <- textGrob(paste0("Library: ", prefix, "\n", 
                        nrow(x = rna), " genes across ", ncol(x = rna), 
                        " cells \n------------------------------------ \n"), 
                        gp = gpar(fontsize = 18, fontface = 'bold'))
        plot_list = list(obj[[1]])
        obj = create_plot(plot_list, tg, sg, width = 8)
        grid.draw(obj)
        printPNG(name = "heatmap", plotObject = obj, papermill = papermill)
        
        # Code end to create heatmap 

        log_print("SUCCESSFUL: Heatmap")
    },
    error = function(cond) {
        log_print("ERROR: Heatmap")
        log_print(cond)
    }
)

In [ ]:
# Jackstraw Plot

tryCatch({
        log_print("Jackstraw Plot")

        # Code start to create jackstraw plot 
    
        rna <- JackStraw(rna, num.replicate = jackstraw_replicates)
        rna <- ScoreJackStraw(rna, dims = 1:jackstraw_score_dim)
        obj <- JackStrawPlot(rna, dims = 1:jackstraw_plot_dim)
    
        tg <- textGrob('Jackstraw Plot', gp = gpar(fontsize = 25, fontface = 'bold', col = 'red'))
        sg <- textGrob(paste0("Library: ", prefix, "\n", 
                        nrow(x = rna), " genes across ", ncol(x = rna), 
                        " cells \n------------------------------------ \n"), 
                        gp = gpar(fontsize = 18, fontface = 'bold'))
        plot_list = list(obj)
        obj = create_plot(plot_list, tg, sg, width = 8)
        grid.draw(obj)
        printPNG(name = "jackstraw", plotObject = obj, papermill = papermill)
            
        # Code end to create jackstraw plot 

        log_print("SUCCESSFUL: Jackstraw Plot")
    },
    error = function(cond) {
        log_print("ERROR: Jackstraw Plot")
        log_print(cond)
    }
)

In [ ]:
# Elbow Plot

tryCatch({
        log_print("Elbow Plot")

        # Code start to create elbow plot 
    
        obj <- ElbowPlot(rna)
        tg <- textGrob('Elbow Plot', gp = gpar(fontsize = 25, fontface = 'bold', col = 'red'))
        sg <- textGrob(paste0("Library: ", prefix, "\n", 
                        nrow(x = rna), " genes across ", ncol(x = rna), 
                        " cells \n------------------------------------ \n"), 
                        gp = gpar(fontsize = 18, fontface = 'bold'))
        plot_list = list(obj)
        obj = create_plot(plot_list, tg, sg, width = 8)
        grid.draw(obj)
        printPNG(name = "elbow", plotObject = obj, papermill = papermill)
                
        # Code end to create elbow plot 

        log_print("SUCCESSFUL: Elbow Plot")
    },
    error = function(cond) {
        log_print("ERROR: Elbow Plot")
        log_print(cond)
    }
)

In [ ]:
# Run UMAP

tryCatch({
        log_print("Run UMAP")

        # Code start to run umap 
    
        rna <- FindNeighbors(rna, dims = 1:umap_dim)
        rna <- FindClusters(rna, resolution = umap_resolution)
        rna <- RunUMAP(rna, dims = 1:umap_dim)
    
        # Code end to run umap 

        log_print("SUCCESSFUL: Run UMAP")
    },
    error = function(cond) {
        log_print("ERROR: Run UMAP")
        log_print(cond)
    }
)

In [ ]:
# UMAP Plot

tryCatch({
        log_print("UMAP Plot")

        # Code start to create UMAP plot 
    
        obj <- DimPlot(rna, reduction = "umap")
    
        tg <- textGrob('UMAP Plot: Default Clustering', gp = gpar(fontsize = 25, fontface = 'bold', col = 'red'))
        sg <- textGrob(paste0("Library: ", prefix, "\n", 
                        nrow(x = rna), " genes across ", ncol(x = rna), 
                        " cells \n------------------------------------ \n"), 
                        gp = gpar(fontsize = 18, fontface = 'bold'))
        plot_list = list(obj)
        obj = create_plot(plot_list, tg, sg, width = 8)
        grid.draw(obj)
        printPNG(name = "umap", plotObject = obj, papermill = papermill)
    
        grid.newpage()
    
        #obj <- FeaturePlot(object = rna, features = "nCount_RNA", reduction = "umap")
        df = as.data.frame(rna[["umap"]]@cell.embeddings)
        df$nCount_RNA = rna$nCount_RNA
    
        obj <- ggplot(data=df) + geom_point(aes(x = UMAP_1, y = UMAP_2, color = log10(nCount_RNA))) + 
        scale_color_viridis() + 
        theme_bw() + 
        theme(plot.title = element_text(hjust = 0.5),panel.border = element_blank(), panel.grid.major = element_blank(), panel.grid.minor = element_blank(), axis.line = element_line(colour = "black"))
        
        tg <- textGrob('UMAP Plot: Color by # of RNA molecules', gp = gpar(fontsize = 25, fontface = 'bold', col = 'red'))
        sg <- textGrob(paste0("Library: ", prefix, "\n", 
                        nrow(x = rna), " genes across ", ncol(x = rna), 
                        " cells \n------------------------------------ \n"), 
                        gp = gpar(fontsize = 18, fontface = 'bold'))
        plot_list = list(obj)
        obj = create_plot(plot_list, tg, sg, width = 8)
        grid.draw(obj)
        printPNG(name = "umap_count_rna", plotObject = obj, papermill = papermill)
    
        grid.newpage()
    
        #obj <- FeaturePlot(object = rna, features = "nFeature_RNA", reduction = "umap") 
        df$nFeature_RNA = rna$nFeature_RNA
    
        obj <- ggplot(data=df) + geom_point(aes(x = UMAP_1, y = UMAP_2, color = log10(nFeature_RNA))) + 
        scale_color_viridis() + 
        theme_bw() + 
        theme(plot.title = element_text(hjust = 0.5),panel.border = element_blank(), panel.grid.major = element_blank(), panel.grid.minor = element_blank(), axis.line = element_line(colour = "black"))
        
        tg <- textGrob('UMAP Plot: Color by # of genes detected', gp = gpar(fontsize = 25, fontface = 'bold', col = 'red'))
        sg <- textGrob(paste0("Library: ", prefix, "\n", 
                        nrow(x = rna), " genes across ", ncol(x = rna), 
                        " cells \n------------------------------------ \n"), 
                        gp = gpar(fontsize = 18, fontface = 'bold'))
        plot_list = list(obj)
        obj = create_plot(plot_list, tg, sg, width = 8)
        grid.draw(obj)
        printPNG(name = "umap_count_gene", plotObject = obj, papermill = papermill)
    
        grid.newpage()
    
        obj <- FeaturePlot(object = rna, features = "percent.mt", reduction = "umap")
        tg <- textGrob('UMAP Plot: Color by mitochondria percent', gp = gpar(fontsize = 25, fontface = 'bold', col = 'red'))
        sg <- textGrob(paste0("Library: ", prefix, "\n", 
                        nrow(x = rna), " genes across ", ncol(x = rna), 
                        " cells \n------------------------------------ \n"), 
                        gp = gpar(fontsize = 18, fontface = 'bold'))
        plot_list = list(obj)
        obj = create_plot(plot_list, tg, sg, width = 8)
        grid.draw(obj)
        printPNG(name = "umap_mito", plotObject = obj, papermill = papermill)
        
        grid.newpage()
    
        # Code end to create UMAP plot 

        log_print("SUCCESSFUL: UMAP Plot")
    },
    error = function(cond) {
        log_print("ERROR: UMAP Plot")
        log_print(cond)
    }
)

In [ ]:
#Create final output files

files2zip <- dir('plots/', full.names = TRUE)
zip(zipfile = 'plots.zip', files = files2zip)

saveRDS(rna, file = paste0(prefix,".rna.seurat.rds.",genome,".rds"))

log_close()